<a href="https://colab.research.google.com/github/bessabra/PRO2PRO/blob/main/cacopospro.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [41]:
import pandas as pd
from datetime import datetime, timedelta

base  = pd.read_excel('/content/CACOPOSPRO v01.01.xlsx')
base2 = pd.read_excel('/content/baseAnalisev09.01.xlsx')


Retirada de colunas desnecessáras.

In [42]:
base2.drop('_merge',        axis = 1, inplace=True)


Redefinição de tipos das variáveis.

In [105]:
colunas_conv_data_base = ['VIGENCIA INICIAL', 
                          'VIGENCIA FINAL']

colunas_conv_data_base2 = ['DATA DO REQUERIMENTO',
                          'OBITO',
                          'INATIVIDADE',
                          'DATA EXCLUSÃO',
                          'DR5']       

#Substituindo
for coluna in colunas_conv_data_base2:
  base2[coluna] = pd.to_datetime(base2[coluna], errors="coerce")

#Substituindo
for coluna in colunas_conv_data_base:
  base[coluna] = pd.to_datetime(base[coluna], errors="coerce")

Filtrando SIAPEs presentes em CACOPOSPRO e baseAnalise

In [106]:
#Merge entre a base e base2
servEncontradosCaco = base.merge(
    base2, on=['SIAPE'], 
    # how='outer',
    how='inner',
    suffixes=['', '_'], 
    indicator=True
    )

#Calculando a diferença entre a VIGENCIA INICIAL do CNR e a DATA DO REQUERIMENTO em dias
servEncontradosCaco['DiasDiffDR'] = (servEncontradosCaco['VIGENCIA INICIAL'] - servEncontradosCaco['DATA DO REQUERIMENTO']).dt.days.abs()

#Ordena por SIAPE a diferença de dias
servEncontradosCacoDR = servEncontradosCaco.sort_values(by=['SIAPE', 'DiasDiffDR'])
servEncontradosCacoDR = servEncontradosCacoDR.groupby('SIAPE').first().reset_index()

#Renomeia as colunas para evitar conflitos no merge
servEncontradosCacoDR = servEncontradosCacoDR.rename(
    columns = {'POSICIONAMENTO':'NivelEmDR'}
    )
servEncontradosCaco = servEncontradosCaco.rename(
    columns = {'_merge':'MergeGeral'}
    )
servEncontradosCacoDR = servEncontradosCacoDR.rename(
    columns = {'_merge':'MergeDR'}
    )

servEncontradosCaco = servEncontradosCaco.rename(
    columns = {'POSICIONAMENTO':'NIVEL POR DATA'}
    )

#Merge entre servEncontradosCaco e servEncontradosCacoDR para identificar o CNR em DR nas proximas etapas do script
servEncontradosCaco = servEncontradosCaco.merge(
    servEncontradosCacoDR, on=['SIAPE'], 
    # how='outer',
    how='inner',
    suffixes=['', '_'], 
    indicator=True
    )


/usr/local/lib/python3.8/dist-packages/pandas/core/frame.py:9190: FutureWarning: Passing 'suffixes' which cause duplicate columns {'CPF_'} in the result is deprecated and will raise a MergeError in a future version.
  return merge(


In [107]:
# display(servEncontradosCaco)

In [108]:
# Dicionário com as informações de mudança
substituicoesCodNivel = { '709-NS-D-101': 1, '707-NS-D-101': 1, 
                          '709-NS-D-102': 2, '707-NS-D-102': 2, 
                          '709-NS-D-201': 3, '707-NS-D-201': 3, 
                          '709-NS-D-202': 4, '707-NS-D-202': 4, 
                          '709-NS-D-301': 5, '707-NS-D-301': 5, 
                          '709-NS-D-302': 6, '707-NS-D-302': 6, 
                          '709-NS-D-303': 7, '707-NS-D-303': 7, 
                          '709-NS-D-304': 8, '707-NS-D-304': 8, 
                          '709-NS-D-401': 9, '707-NS-D-401': 9,
                          '709-NS-D-402':10, '707-NS-D-402': 10, 
                          '709-NS-D-403':11, '707-NS-D-403': 11, 
                          '709-NS-D-404':12, '707-NS-D-404': 12,
                          '704-NS-D-101': 1, '702-NA-D-101': 1, 
                          '704-NS-D-102': 2, '702-NA-D-102': 2, 
                          '704-NS-D-201': 3, '702-NA-D-201': 3,
                          '704-NS-D-202': 4, '702-NA-D-202': 4, 
                          '704-NS-D-301': 5, '702-NA-D-301': 5, 
                          '704-NS-D-302': 6, '702-NA-D-302': 6,
                          '704-NS-D-303': 7, '702-NA-D-303': 7, 
                          '704-NS-D-304': 8, '702-NA-D-304': 8,
                          '704-NS-D-401': 9, '702-NA-D-401': 9, 
                          '704-NS-D-402':10, '702-NA-D-402':10,
                          '704-NS-D-403':11, '702-NA-D-403':11, 
                          '704-NS-D-404':12, '702-NA-D-404':12}

#Substituindo
for valor_antigo, valor_novo in substituicoesCodNivel.items():
    servEncontradosCaco.loc[servEncontradosCaco['NivelEmDR'] == valor_antigo, 'COD_NIVELDR'] = valor_novo

# display(servEncontradosCaco)

In [109]:
servEncontradosCaco.loc[
    (servEncontradosCaco['VIGENCIA INICIAL_'] > servEncontradosCaco['DATA DO REQUERIMENTO']), 'COD_NIVELDR'] = servEncontradosCaco['COD_NIVELDR'] - 1

In [110]:
# Dicionário com as informações de mudança
substituicoesCodNivel = { 1:101,  2:102, 
                          3:201,  4:202, 
                          5:301,  6:302, 
                          7:303,  8:304, 
                          9:401,  10:402, 
                          11:403, 12:404}

#Substituindo
for valor_antigo, valor_novo in substituicoesCodNivel.items():
    servEncontradosCaco.loc[servEncontradosCaco['COD_NIVELDR'] == valor_antigo, 'NIVEL_DR'] = valor_novo

# display(servEncontradosCaco)

Calculando a diferença entre DR5 e as datas de VIGENCIA INICIAL de cada Código de Carreira e Classe, por servidor. 

Em seguida, uma nova base é criada filtrada com a data e o Código de Carreira e Classe de cada servidor.

In [111]:
servEncontradosCaco['DiasDiff'] = (servEncontradosCaco['VIGENCIA INICIAL'] - servEncontradosCaco['DR5']).dt.days.abs()

servEncontradosCaco = servEncontradosCaco.sort_values(by=['NOME', 'DiasDiff'])
servEncontradosCaco = servEncontradosCaco.groupby('NOME').first().reset_index()

Calculando (DTPi - DR5), para identificar o valor da diferença e filtrar a decisão.

In [112]:
servEncontradosCaco.loc[
    ((servEncontradosCaco['VIGENCIA INICIAL'].dt.year - servEncontradosCaco['DR5'].dt.year <= 1) 
    & ((servEncontradosCaco['CARGO'] == '709001 PROFESSOR DO ENSINO BASICO EX-TERRITORIO')|(servEncontradosCaco['CARGO'] == '704001 PROFESSOR DO ENSINO BASICO EX-TERRITORIO'))), 'REGRA'] = 'EBF <= 1'

servEncontradosCaco.loc[
    ((servEncontradosCaco['VIGENCIA INICIAL'].dt.year - servEncontradosCaco['DR5'].dt.year > 1) 
    & ((servEncontradosCaco['CARGO'] == '709001 PROFESSOR DO ENSINO BASICO EX-TERRITORIO')|(servEncontradosCaco['CARGO'] == '704001 PROFESSOR DO ENSINO BASICO EX-TERRITORIO'))), 'REGRA'] = 'EBF > 1' 

servEncontradosCaco.loc[
     ((servEncontradosCaco['VIGENCIA INICIAL'].dt.year - servEncontradosCaco['DR5'].dt.year <= 1) 
     & ((servEncontradosCaco['CARGO'] == '707001 PROFESSOR ENS BASICO TECN TECNOL')|(servEncontradosCaco['CARGO'] == '702001 PROF DO ENSINO BASICO TEC,TECNOLOGICO'))), 'REGRA'] = 'EBTT <= 1'

servEncontradosCaco.loc[
     ((servEncontradosCaco['VIGENCIA INICIAL'].dt.year - servEncontradosCaco['DR5'].dt.year > 1) 
     & ((servEncontradosCaco['CARGO'] == '707001 PROFESSOR ENS BASICO TECN TECNOL') |(servEncontradosCaco['CARGO'] == '702001 PROF DO ENSINO BASICO TEC,TECNOLOGICO'))), 'REGRA'] = 'EBTT > 1'

servEncontradosCaco.loc[
     ((servEncontradosCaco['CARGO'] != '707001 PROFESSOR ENS BASICO TECN TECNOL') &
      (servEncontradosCaco['CARGO'] != '702001 PROF DO ENSINO BASICO TEC,TECNOLOGICO') &
      (servEncontradosCaco['CARGO'] != '709001 PROFESSOR DO ENSINO BASICO EX-TERRITORIO') &
      (servEncontradosCaco['CARGO'] != '704001 PROFESSOR DO ENSINO BASICO EX-TERRITORIO')), 'REGRA'] = 'MUDANÇA DE CARREIRA'

Tabelas de conversão de Código de Carreira e Classe (object) para um código de valor entre 1 - 12 (inteiro), isso facilitará o incremento.

In [113]:
# Dicionário com as informações de mudança
substituicoesCodNivel = { '709-NS-D-101': 1, '707-NS-D-101': 1, 
                          '709-NS-D-102': 2, '707-NS-D-102': 2, 
                          '709-NS-D-201': 3, '707-NS-D-201': 3, 
                          '709-NS-D-202': 4, '707-NS-D-202': 4, 
                          '709-NS-D-301': 5, '707-NS-D-301': 5, 
                          '709-NS-D-302': 6, '707-NS-D-302': 6, 
                          '709-NS-D-303': 7, '707-NS-D-303': 7, 
                          '709-NS-D-304': 8, '707-NS-D-304': 8, 
                          '709-NS-D-401': 9, '707-NS-D-401': 9,
                          '709-NS-D-402':10, '707-NS-D-402': 10, 
                          '709-NS-D-403':11, '707-NS-D-403': 11, 
                          '709-NS-D-404':12, '707-NS-D-404': 12,
                          '704-NS-D-101': 1, '702-NA-D-101': 1, 
                          '704-NS-D-102': 1, '702-NA-D-102': 1,
                          '704-NS-D-103': 2, '702-NA-D-103': 2,
                          '704-NS-D-104': 2, '702-NA-D-104': 2,      
                          '704-NS-D-201': 3, '702-NA-D-201': 3,
                          '704-NS-D-202': 3, '702-NA-D-202': 3, 
                          '704-NS-D-203': 4, '702-NA-D-203': 4,
                          '704-NS-D-204': 4, '702-NA-D-204': 4, 
                          '704-NS-D-301': 5, '702-NA-D-301': 5, 
                          '704-NS-D-302': 6, '702-NA-D-302': 6,
                          '704-NS-D-303': 7, '702-NA-D-303': 7, 
                          '704-NS-D-304': 8, '702-NA-D-304': 8,
                          '704-NS-D-401': 9, '702-NA-D-401': 9, 
                          '704-NS-D-402':10, '702-NA-D-402':10,
                          '704-NS-D-403':11, '702-NA-D-403':11, 
                          '704-NS-D-404':12, '702-NA-D-404':12}

#Substituindo
for valor_antigo, valor_novo in substituicoesCodNivel.items():
    servEncontradosCaco.loc[servEncontradosCaco['NIVEL POR DATA'] == valor_antigo, 'COD_NIVEL_DR5'] = valor_novo

# display(servEncontradosCaco)

In [114]:
servEncontradosCaco.loc[
    ((servEncontradosCaco['VIGENCIA INICIAL'] > servEncontradosCaco['DR5'])&(servEncontradosCaco['COD_NIVEL_DR5'] > 1)), 'COD_NIVEL_DR5'] = servEncontradosCaco['COD_NIVEL_DR5'] - 1

Realização do incremento de acordo com o filtro da diferença entre DTPi - DR5.


In [115]:
servEncontradosCaco.loc[
    (servEncontradosCaco['REGRA'] == 'EBF <= 1'), 'INCREMENTO_CNR'] = servEncontradosCaco['COD_NIVEL_DR5'] + 3
servEncontradosCaco.loc[
    (servEncontradosCaco['REGRA'] == 'EBF > 1'), 'INCREMENTO_CNR'] = servEncontradosCaco['COD_NIVEL_DR5'] + 4


servEncontradosCaco.loc[
    (servEncontradosCaco['REGRA'] == 'EBTT <= 1'), 'INCREMENTO_CNR'] = servEncontradosCaco['COD_NIVEL_DR5'] + 2
servEncontradosCaco.loc[
    (servEncontradosCaco['REGRA'] == 'EBTT > 1'), 'INCREMENTO_CNR'] = servEncontradosCaco['COD_NIVEL_DR5'] + 3

Conversão do código de valores incrementados de acordo com a regra de diferença entre DTPi - DR5.

In [116]:
substituicoesCNR = { 1:101,  2:102, 
                     3:201,  4:202, 
                     5:301,  6:302, 
                     7:303,  8:304, 
                     9:401,  10:402, 
                     11:403}

# Substituindo valores menores que 12
for valor_antigo, valor_novo in substituicoesCNR.items():
    servEncontradosCaco.loc[servEncontradosCaco['INCREMENTO_CNR'] == valor_antigo, 'CNR'] = valor_novo
    
# Substituindo valores iguais ou maiores que 12
servEncontradosCaco.loc[servEncontradosCaco['INCREMENTO_CNR'] >= 12, 'CNR'] = 404

# display(servEncontradosCaco)

Descrição do resultado da análise.

In [117]:
servEncontradosCaco.loc[
    ((servEncontradosCaco['CNR'] < servEncontradosCaco['NIVEL_DR'])|
     (servEncontradosCaco['CNR'] > servEncontradosCaco['NIVEL_DR'])|
     (servEncontradosCaco['REGRA'] == 'MUDANÇA DE CARREIRA')), 'RESULTADO'] = 'A DECIDIR'
servEncontradosCaco.loc[
    (servEncontradosCaco['CNR'] == servEncontradosCaco['NIVEL_DR']), 'RESULTADO'] = 'INDEFERIDO'

servEncontradosCaco.loc[
    (servEncontradosCaco['CNR'] < servEncontradosCaco['NIVEL_DR']), 'CNRstatus'] = 'O CNR calculado é MENOR que o encontrado na Data do Requerimento'
servEncontradosCaco.loc[
    (servEncontradosCaco['CNR'] > servEncontradosCaco['NIVEL_DR']), 'CNRstatus'] = 'O CNR calculado é MAIOR que o encontrado na Data do Requerimento'
servEncontradosCaco.loc[
    (servEncontradosCaco['CNR'] == servEncontradosCaco['NIVEL_DR']), 'CNRstatus'] = 'O CNR calculado é IGUAL ao encontrado na Data do Requerimento'

In [118]:
colunas_conv_data = ['DATA DO REQUERIMENTO', 
                     'VIGENCIA INICIAL', 
                     'VIGENCIA FINAL',
                     'OBITO',
                     'INATIVIDADE',
                     'DATA EXCLUSÃO']       

#Substituindo
for coluna in colunas_conv_data:
  servEncontradosCaco[coluna] = pd.to_datetime(servEncontradosCaco[coluna]).dt.strftime('%d/%m/%Y')

# display(servEncontradosCaco['VIGENCIA INICIAL'])

In [ ]:
# servEncontradosCaco['CPF'] = servEncontradosCaco['CPF'].astype(str).str.pad(11, fillchar='0', side='left').astype(int)
# display(servEncontradosCaco['CPF'])

Geração do xlsx com os resultados.

In [122]:
# Lista de colunas para manter
colunas_manter = ['PROCESSO SEI', 
                  'DATA DO REQUERIMENTO', 
                  'SIAPE', 
                  'INTERESSADO',
                  'NIVEL POR DATA',
                  'VIGENCIA INICIAL',
                  'VIGENCIA FINAL',
                  'SITUAÇÃO',
                  'SERVIDOR',
                  'CPF',
                  'OBITO', 
                  'INATIVIDADE',
                  'CARGO',
                  'EXCLUSÃO',
                  'DATA EXCLUSÃO',
                  'SITUAÇÃO CADASTRO',
                  'ESTADO',
                  'DR5',
                  'NivelEmDR',
                  'RESULTADO',
                  'CNRstatus']              											 				

# Exclusão das colunas que não estão na lista
servEncontradosCaco = servEncontradosCaco[colunas_manter]

servEncontradosCaco.to_excel('ResultadoComplexo.xlsx')